## 구글 드라이브 마운트
colab 런타임을 구글 드라이브에 마운트시킨 후, 적절한 경로를 이동

In [ ]:
import os
from google.colab import drive

drive.mount("/content/drive", force_remount=True)

# 경로로 이동
path = '*******************************************'
os.chdir(path)

In [ ]:
os.listdir()

## 환경 설정
KorBERT를 활용할 수 있는 환경을 설정. 우선 아래와 같은 모듈들을 설치.
  - tensorflow-gpu==0.15
  - tensorflow-hub==0.7.0


그 후, 사용 가능한 gpu를 환경변수 CUDA_VISIBLE_DEVICES 변수에 설정.

In [ ]:
!pip install tensorflow-gpu==1.15
!pip install tensorflow-hub==0.7.0

사용 가능한 gpu를 확인. 만일 런타임 유형 변경에서 가속기로 gpu를 설정하지 않는다면 다음과 같은 에러 메시지가 출력.

`NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.`

이럴 경우 런타임 유형을 변경한 후 다시 실행.

In [ ]:
!export CUDA_VISIBLE_DEVICES=0 # 0번 gpu
!nvidia-smi 

# checkpoint -> keras layer
ETRI의 KorBERT를 keras model로 변환. 이는 `bert_to_keras.py`로 가능.

In [ ]:
# bert_module로 변환해 줬으면 이 과정은 생략 가능.
# kobert -> 케라스 레이어로 변환
!python bert_to_keras.py 

In [ ]:
# 'bert-module'이라는 경로가 추가적으로 생성된 것을 확인 가능.
os.listdir()

## 데이터 전처리
KorBERT가 Fine-Tune을 할 수 있게끔 데이터를 준비해야 함. 준비해야 하는 데이터 파일은 tab(\t)을 기준으로 label과 input sequence가 나뉘어 있어야 한다.

`prepare_data.py`를 실행할 경우 data -> label, seq.in 으로 나뉘게 되는데, 이떄 seq.in은 KorBERT가 pre-train된 대로 tokenization 과정을 거친다.

`prepare_data.py`의 경우 mandatory한 인자로 --input(-i)와 --output(-o)를 갖는다.

`python prepare_data.py --input ${input_data} --output ${processed_data_path}`


In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd

data = pd.read_csv('./data/data_sample.txt', sep = '\t', header = None, names = [0, 1], encoding = 'utf-8')

# test의 비율을 0.2(20%)로 설정
X_train, X_test, y_train, y_test = train_test_split(data[1], data[0], test_size=0.2, random_state=42)
# test에서 0.5의 비율로 훈련데이터와 검증데이터를 반반씩 나누도록 설정
X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

train = pd.DataFrame({0:y_train, 1:X_train})
valid = pd.DataFrame({0:y_valid, 1:X_valid})
test = pd.DataFrame({0:y_test, 1:X_test})

# train(훈련데이터), valid(검증데이터), test(테스트데이터)로 파일을 각각 생성
train.to_csv('/data/train.txt', sep='\t', header=None, index=None)
test.to_csv('/data/test.txt', sep='\t', header=None, index=None)
valid.to_csv('/data/valid.txt', sep='\t', header=None, index=None)

In [ ]:
# !python prepare_data.py --input data/data_sample.txt --output data/
!python prepare_data.py --input data/train.txt --output data/train/
!python prepare_data.py --input data/valid.txt --output data/valid/
!python prepare_data.py --input data/test.txt --output data/test/

KorBERT의 tokenization 방식을 알기 위해서는 다음 코드와

In [ ]:
from vectorizers import tokenizationK as tk


sample_sentence = '와 이 커피 진짜 맛있다.'
vocab_file_path = '*******************************************'

tokenizer = tk.FullTokenizer(vocab_file_path)

tokens = ""

for word in sample_sentence.split():
    word_tokens = " ".join(tokenizer.tokenize(word))
    tokens += word_tokens + " "

In [ ]:
tokens

'와_ 이_ 커 피_ 진짜_ 맛 있다 ._ '

## Fine-Tuning 훈련

준비된 데이터를 바탕으로 KorBERT fine-tuning을 실시.

`train_bert_finetuning.py`를 실행하며, mandatory 인자로는 --train(-t) 와 --save(-s)를 받는다.

`python train_bert_finetuning.py --train ${processed_data_path} --save ${saved_model}`

In [ ]:
!python train_bert_finetuning.py --train ******************** --val ********************  --save ******************** --epochs 10 --batch 10 --type 'bert'

## EVALUATION

In [ ]:
!python eval_bert_finetuning.py --model ******************** --data ********************